# 1- Imports

In [7]:
import sqlite3
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.stats.proportion as prop

# 2 - Conexao com o banco

In [8]:

    # conexao com o banco e query
    conn = sqlite3.connect('/Users/User/Documents/repos/case_workana/dastabase.sqlite')
    query = '''

    with tb_accepted_bids as (
    -- cte com os bids aceitos e as datas para filtro (as datas vem da tabela de bids)
        select 
            a.bid_id,
            b.project_id, 
            1 as flg_accepted_bid,
            b.created_date
        from 
            accepted_bids as a
        left join 
            bids as b on a.bid_id = b.id
        where 
            1=1
            and b.created_date between '03-07-2025' and '21-07-2025' 
            and a.status = 'active'
            and b.id is not null 
    )

    -- cte final que traz os bids aceitos para os project_id da base do teste
    select
        a.id,
        a.project_id,
        a.name,
        a.segment,
        a.created as data_teste,
        b.*
    from 
        abtests as a
    left join 
        tb_accepted_bids as b on a.project_id = b.project_id  


    '''



# 3 - Criaçao de df e teste

In [9]:
#%% 

# criacao do df
df = pd.read_sql(query, con=conn)
df_teste = df[['segment', 'bid_id']]

##############################################################################################################

## aplicaçao do teste z de proporcoes 
## amostras grandes o suficiente
## nao é necessario testar normalidade porque o teste ja se aproxima da distribuiçao normal
## apenas dois grupos (se fosse mais seria chi quadrado)



# Hipotese nula: a proporçao de sucesso do grupo de teste é menor ou igual a proporçao do grupo controle (status quo)
# Hipotese alternativa:  a proporçao de sucesso do grupo de teste é maior do que a proporçao do grupo controle

# Hnull : P1 <= P2
# Halt : P1 > P2 

# Onde P1 é evalbidsNewOrder e P2 é default


#############################################################################################################

df_teste['bid_aceito'] = df_teste['bid_id'].notna()

# agrupamento por base teste e controle
contagens = df_teste.groupby('segment')['bid_aceito'].agg(['count', 'sum']).reset_index()
contagens.columns = ['segment', 'total', 'sucessos']

segmentos_para_teste = contagens.head(2) 

# criacao de listas com os dados sucesso e de cada grupo para o teste

p1 = contagens.loc[contagens['segment'] == 'evalbidsNewOrder', 'sucessos'].iloc[0]
p2 = contagens.loc[contagens['segment'] == 'default', 'sucessos'].iloc[0]

p1_total_amostra = contagens.loc[contagens['segment'] == 'evalbidsNewOrder', 'total'].iloc[0]
p2_total_amostra = contagens.loc[contagens['segment'] == 'default', 'total'].iloc[0]
#%%

sucessos = [p1,p2]
totais = [p1_total_amostra, p2_total_amostra]


# aplicacao do teste
stat, pvalor = prop.proportions_ztest(count=sucessos, nobs=totais, alternative='larger' )    


# calculo intervalo de confiança

# calculo de proporçoes
p1_hat = p1 / p1_total_amostra
p2_hat = p2 / p2_total_amostra
diff = p1_hat - p2_hat

# tamanho da amostra

# z score (para 95% de confiança é 1.96)
z_score = 1.96

# calculo da margem de erro
margem_de_erro_p1 = z_score * np.sqrt(p1_hat*(1-p1_hat)/p1_total_amostra)
margem_de_erro_p2 = z_score * np.sqrt(p2_hat*(1-p2_hat)/p2_total_amostra)

margem_de_erro_diff = z_score * np.sqrt((p1_hat * (1 - p1_hat) / p1_total_amostra) + (p2_hat * (1 - p2_hat) / p2_total_amostra))

# calculo IC
lower_bound_p1 = p1_hat - margem_de_erro_p1 
upper_bound_p1 = p1_hat + margem_de_erro_p1

lower_bound_p2 = p2_hat - margem_de_erro_p2
upper_bound_p2 = p2_hat + margem_de_erro_p2

lower_bound_diff = diff - margem_de_erro_diff
upper_bound_diff = diff + margem_de_erro_diff


print(f"Teste of difference in proportions: {df['segment'].unique()[0]} vs {df['segment'].unique()[1]}")
print(f"Z = {stat:.4f}, p = {pvalor:.4f}")
print(f"Significative: {'YES' if pvalor < 0.05 else 'NÃO'}")
print(f'IC diff between groups {lower_bound_diff}, {upper_bound_diff}')
#print(f'IC grupo teste {lower_bound_p1}, {upper_bound_p1}')
#print(f'IC grupo controle {lower_bound_p2}, {upper_bound_p2}')





Teste of difference in proportions: evalbidsNewOrder vs default
Z = 1.7130, p = 0.0434
Significative: YES
IC diff between groups -0.00349914172178966, 0.0518328641974533


C:\Users\User\AppData\Local\Temp\ipykernel_9188\1289548011.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['bid_aceito'] = df_teste['bid_id'].notna()


## 4 - Calculo de novo tamanho de amostra para teste posterior

In [10]:
significance_95_percent = 1.96
statistical_power_80_percent = 0.8 # default do mercado
mde = 0
baseline_conversion_rate = p2 / p2_total_amostra
test_conversion_rate = p1 / p1_total_amostra
mde = test_conversion_rate - baseline_conversion_rate
pooled_cr = (test_conversion_rate + baseline_conversion_rate) / 2

numerator = (significance_95_percent * np.sqrt(2 * pooled_cr * (1 - pooled_cr)) + 
             statistical_power_80_percent * np.sqrt(test_conversion_rate*(1-test_conversion_rate) + baseline_conversion_rate*(1-baseline_conversion_rate)))**2
denominator = mde**2

sample_size_per_group = numerator / denominator



print(baseline_conversion_rate)
print(test_conversion_rate)
print(mde)
print(pooled_cr)
print(f"Sample size needed per group: {sample_size_per_group}") 
print(f'Total sample sixe {(2 * sample_size_per_group).astype(int)}')


0.14580265095729014
0.16996951219512196
0.02416686123783182
0.15788608157620604
Sample size needed per group: 3467.2412971367894
Total sample sixe 6934


### BACKUP

In [11]:
query2 = ''' with tb_accepted_bids as (
    -- cte com os bids aceitos e as datas para filtro (as datas vem da tabela de bids)
        select 
            a.bid_id,
            b.project_id, 
            1 as flg_accepted_bid,
            b.created_date,
			case when b.usd_amount is null then 0 else b.usd_amount end as usd_amount
        from 
            accepted_bids as a
        left join 
            bids as b on a.bid_id = b.id
        where 
            1=1
            and b.created_date between '03-07-2025' and '21-07-2025' 
            and a.status = 'active'
            and b.id is not null 
    )

    -- cte final que traz os bids aceitos para os project_id da base do teste
    select
        a.id,
        a.project_id,
        a.name,
        a.segment,
        a.created as data_teste,
        b.*
    from 
        abtests as a
    left join 
        tb_accepted_bids as b on a.project_id = b.project_id  

'''

In [13]:
# %% 

# criacao do df (mantido)
df = pd.read_sql(query2, con=conn)
# O dataframe de teste conterá a coluna 'usd_amount' (o valor) e 'segment' (o grupo)
df_teste = df[['segment', 'usd_amount']].copy()

# ==============================================================================
# 💡 CORREÇÃO: Converter a coluna 'usd_amount' para numérica
# O parâmetro 'coerce' transforma valores não numéricos (como strings vazias ou erros de leitura) em NaN.
# ==============================================================================
df_teste['usd_amount'] = pd.to_numeric(df_teste['usd_amount'], errors='coerce')


##############################################################################################################

## aplicaçao do teste t de diferença de médias para amostras independentes
## Assumimos que a coluna 'usd_amount' representa a variável numérica de interesse.
## Vamos testar a diferença entre as médias dos valores de usd_amount para os dois segmentos.

# Hipótese nula (H0): A média do grupo de teste é menor ou igual à média do grupo controle (μ1 <= μ2)
# Hipótese alternativa (Ha): A média do grupo de teste é maior do que a média do grupo controle (μ1 > μ2)
# Onde μ1 é 'evalbidsNewOrder' e μ2 é 'default'

from scipy import stats
import numpy as np

# 1. Separar os dados em dois grupos
# O .dropna() aqui é crucial, pois a conversão 'coerce' pode ter criado NaN's
grupo_teste = df_teste[df_teste['segment'] == 'evalbidsNewOrder']['usd_amount'].dropna()
grupo_controle = df_teste[df_teste['segment'] == 'default']['usd_amount'].dropna()

# 2. Verificar a premissa de igualdade de variâncias (teste de Levene)
# É necessário garantir que haja dados suficientes em AMBOS os grupos para o teste de Levene
if len(grupo_teste) < 2 or len(grupo_controle) < 2:
    print("\nERRO: Pelo menos um dos grupos tem menos de 2 observações após remover NA's. Não é possível rodar o Teste t.")
else:
    levene_test = stats.levene(grupo_teste, grupo_controle)
    equal_variances = levene_test.pvalue > 0.05

    print(f"Teste de Levene: p = {levene_test.pvalue:.4f}. Assumindo variâncias iguais: {equal_variances}")

    # 3. Aplicação do teste t (unilateral 'greater')
    # 'greater' testa se a média do primeiro array (grupo_teste) é maior que a média do segundo (grupo_controle)
    stat, pvalor = stats.ttest_ind(
        a=grupo_teste, 
        b=grupo_controle, 
        equal_var=equal_variances, 
        alternative='greater'
    )    

    # 4. Cálculo do Intervalo de Confiança para a Diferença de Médias
    # Média, desvio padrão e tamanho da amostra
    mean1, std1, n1 = grupo_teste.mean(), grupo_teste.std(ddof=1), len(grupo_teste)
    mean2, std2, n2 = grupo_controle.mean(), grupo_controle.std(ddof=1), len(grupo_controle)
    diff_means = mean1 - mean2
    confidence_level = 0.95 

    # Cálculo dos graus de liberdade e erro padrão conforme a suposição de variâncias (Welch/Student)
    if equal_variances:
        # Fórmula de Student
        pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
        se_diff_ic = pooled_std * np.sqrt(1/n1 + 1/n2)
        df_ic = n1 + n2 - 2
    else:
        # Fórmula de Welch (Satterthwaite)
        se_diff = np.sqrt((std1**2 / n1) + (std2**2 / n2))
        se_diff_ic = se_diff
        # Para evitar problemas de arredondamento em casos de 0 no denominador ou n < 2, 
        # é recomendado usar uma fórmula mais robusta ou o df retornado por ttest_ind se disponível (Scipy > 1.11)
        # Manteremos a estimativa de Satterthwaite, mas com a checagem inicial de n.
        df_ic = (se_diff**4) / ( (std1**2 / n1)**2 / (n1 - 1) + (std2**2 / n2)**2 / (n2 - 1) )
        
    # T-score para 95% (bilateral)
    t_score = stats.t.ppf(1 - (1 - confidence_level)/2, df=df_ic)

    # Margem de erro e IC
    margem_de_erro_diff = t_score * se_diff_ic
    lower_bound_diff = diff_means - margem_de_erro_diff
    upper_bound_diff = diff_means + margem_de_erro_diff


    print("\n" + "="*70)
    print(f"Resultado do Teste t de Diferença de Médias (Unilateral: μ1 > μ2)")
    print("="*70)
    print(f"Grupo Teste ('evalbidsNewOrder') Média: {mean1:.4f}, N: {n1}")
    print(f"Grupo Controle ('default') Média: {mean2:.4f}, N: {n2}")
    print(f"Diferença de Médias (Teste - Controle): {diff_means:.4f}")
    print(f"Estatística T = {stat:.4f}, p-valor = {pvalor:.4f}")
    print(f"Significativo (p < 0.05, Unilateral): {'SIM' if pvalor < 0.05 else 'NÃO'}")
    print(f"IC 95% para a Diferença de Médias: [{lower_bound_diff:.4f}, {upper_bound_diff:.4f}]")
    print("="*70)

Teste de Levene: p = 0.9272. Assumindo variâncias iguais: True

Resultado do Teste t de Diferença de Médias (Unilateral: μ1 > μ2)
Grupo Teste ('evalbidsNewOrder') Média: 140.2500, N: 68
Grupo Controle ('default') Média: 148.2264, N: 53
Diferença de Médias (Teste - Controle): -7.9764
Estatística T = -0.1314, p-valor = 0.5522
Significativo (p < 0.05, Unilateral): NÃO
IC 95% para a Diferença de Médias: [-128.1927, 112.2399]
